In [1]:
from os import getenv
from dotenv import load_dotenv
import json
from services.GoogleSheetsService import *
from services.GeminiService import *
from utils.utils import *
import pandas as pd
from utils.constants import *

In [2]:
load_dotenv('../credentials/.env')

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = getenv("SHEET_ID")
SHEET_PAGE_INVESTMENTS = getenv("RANGE_STRING_INVESTMENTS")
SHEET_PAGE_APPLICATIONS = getenv("RANGE_STRING_APPLICATIONS")
SHEET_PAGE_RETURNS = getenv("RANGE_STRING_RETURNS")
GEMINI_API_KEY = getenv("GEMINI_API_KEY")

In [3]:
sheetService = GoogleSheetsService(SAMPLE_SPREADSHEET_ID)

investments = sheetService.get_sheet(SHEET_PAGE_INVESTMENTS).drop('bank', axis=1)
applications = sheetService.get_sheet(SHEET_PAGE_APPLICATIONS).drop('id', axis=1)
returns = sheetService.get_sheet(SHEET_PAGE_RETURNS).drop('bank', axis=1).drop('id', axis=1)

In [4]:
ids = investments['investment_name'].values

In [5]:
raw_body = {
  'investments': [],
  'applications': [],
  'returns': []
}
for id in ids[3:6]:
  raw_body['investments'].extend(
    restructure_dataframe(
      investments[investments["investment_name"] == id]
    )
  )

  raw_body['applications'].extend(
    restructure_dataframe(
      applications[applications["investment_name"] == id]
    )
  )

  raw_body['returns'].extend(
    restructure_dataframe(
      returns[returns["investment_name"] == id]
    )
  )

print(raw_body)
body_string = f'''Take this as base for the following questions: {json.dumps(raw_body)}'''

{'investments': [{'id': 'INTER CONSERVADOR FIRF CP', 'cycles': [{'value': '887.25', 'date': '2023-12-17'}]}, {'id': 'INTER CORPORATE FIRF CP', 'cycles': [{'value': '336.89', 'date': '2023-12-17'}]}, {'id': 'BUTIA TOP CRÉDITO PRIVADO FIC FIRF', 'cycles': [{'value': '565.46', 'date': '2023-12-17'}]}], 'applications': [{'id': 'INTER CONSERVADOR FIRF CP', 'cycles': [{'value': '500', 'date': '2024-01-31'}]}, {'id': 'INTER CORPORATE FIRF CP', 'cycles': []}, {'id': 'BUTIA TOP CRÉDITO PRIVADO FIC FIRF', 'cycles': [{'value': '1000', 'date': '2024-02-29'}, {'value': '1000', 'date': '2024-03-12'}]}], 'returns': [{'id': 'INTER CONSERVADOR FIRF CP', 'cycles': [{'value': '887.49', 'date': '2023-12-20'}, {'value': '890.25', 'date': '2023-12-29'}, {'value': '896.02', 'date': '2024-01-18'}, {'value': '899.84', 'date': '2024-01-31'}]}, {'id': 'INTER CORPORATE FIRF CP', 'cycles': [{'value': '337.19', 'date': '2023-12-20'}, {'value': '338.08', 'date': '2023-12-29'}, {'value': '340.08', 'date': '2024-01-18

In [6]:
chat_bot = GeminiService(GEMINI_API_KEY)
chat_bot.send_message(BASE_PROMPT)
chat_bot.send_message(body_string)

'**INTER CONSERVADOR FIRF CP**\n\n* Cycle of date "2024-01-18":\n    * Gross profit: 896.02 - 890.25 = 5.77\n    * Profit in percentage: (5.77) / (890.25 + 500) * 100 = 0.65%\n\n* Cycle of date "2024-01-31":\n    * Gross profit: 899.84 - 896.02 = 3.82\n    * Profit in percentage: (3.82) / (896.02 + 500) * 100 = 0.43%\n\n**INTER CORPORATE FIRF CP**\n\n* No applications were made to this fund, so there is no profit.\n\n**BUTIA TOP CRÉDITO PRIVADO FIC FIRF**\n\n* Cycle of date "2024-01-18":\n    * Gross profit: 571.26 - 567.71 - 1000 = -438.45\n    * Profit in percentage: (-438.45) / (567.71 + 1000) * 100 = -33.33%\n\n* Cycle of date "2024-01-31":\n    * Gross profit: 573.98 - 571.26 - 1000 = -426.28\n    * Profit in percentage: (-426.28) / (571.26 + 1000) * 100 = -32.28%'

In [7]:
#Melhorando a visualização
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for message in chat_bot.get_history():
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')

> **user**: 
> to calculate what investiment is the best, we need to calculate the profit of each cycle of records. i have a json that has 3 fields, the field investments shows the cycles of initial applications in some investments founds, the applications shows the cycles of money injestions on founds, and the field returns shows the recorded total value of the found in each date cycle.
> 
> To calculate the gross profit in a return record, we need to take the value of the return in the date, and subtract the return in last date of this found, and subtract the applied money between this period
> 
> The json:
> {
>   "investments": [
>     {
>       "id": "BUTIA TOP CREDITO PRIVADO FIC FIRF",
>       "cycles": [
>         {
>           "value": "565.46",
>           "date": "2023-12-17"
>         }
>       ]
>     },
>     {
>       "id": "BTG PACTUAL HEDGE FUNDO DE INVESTIMENTO RENDA",
>       "cycles": [
>         {
>           "value": "2189.77",
>           "date": "2023-12-17"
>         }
>       ]
>     }
>   ],
>   "applications": [
>     {
>       "id": "BUTIA TOP CREDITO PRIVADO FIC FIRF",
>       "cycles": [
>         {
>           "value": "1000",
>           "date": "2024-02-29"
>         },
>         {
>           "value": "1000",
>           "date": "2024-03-12"
>         }
>       ]
>     },
>     {
>       "id": "BTG PACTUAL HEDGE FUNDO DE INVESTIMENTO RENDA",
>       "cycles": [
>         {
>           "value": "1000",
>           "date": "2023-12-29"
>         },
>         {
>           "value": "500",
>           "date": "2024-03-12"
>         }
>       ]
>     }
>   ],
>   "returns": [
>     {
>       "id": "BUTIA TOP CREDITO PRIVADO FIC FIRF",
>       "cycles": [
>         {
>           "value": "565.96",
>           "date": "2023-12-20"
>         },
>         {
>           "value": "567.71",
>           "date": "2023-12-29"
>         },
>         {
>           "value": "571.26",
>           "date": "2024-01-18"
>         },
>         {
>           "value": "573.98",
>           "date": "2024-01-31"
>         }
>       ]
>     },
>     {
>       "id": "BTG PACTUAL HEDGE FUNDO DE INVESTIMENTO RENDA",
>       "cycles": [
>         {
>           "value": "2193.21",
>           "date": "2023-12-20"
>         },
>         {
>           "value": "2202.08",
>           "date": "2023-12-29"
>         },
>         {
>           "value": "3219.70",
>           "date": "2024-01-18"
>         },
>         {
>           "value": "3236.95",
>           "date": "2024-01-31"
>         }
>       ]
>     }
>   ]
> }
> 
> 
> the profit on the cycle of date "2024-01-18" of "BTG PACTUAL HEDGE FUNDO DE INVESTIMENTO RENDA" is equal to 3219.70-2202.08-1000. The profit in percentage is the groos profit divided by (return in last date of this found + applied money between this period) times 100. in this case, is )(3219.70-2202.08-1000)/(2202.08+1000))*100. Use this to calculate what whas requested after.


-------------------------------------------


> **model**: **BUTIA TOP CREDITO PRIVADO FIC FIRF**
> 
> * Cycle of date "2024-01-18":
>     * Gross profit: 571.26 - 567.71 - 1000 = -438.45
>     * Profit in percentage: (-438.45) / (567.71 + 1000) * 100 = -33.33%
> 
> * Cycle of date "2024-01-31":
>     * Gross profit: 573.98 - 571.26 - 1000 = -426.28
>     * Profit in percentage: (-426.28) / (571.26 + 1000) * 100 = -32.28%
> 
> **BTG PACTUAL HEDGE FUNDO DE INVESTIMENTO RENDA**
> 
> * Cycle of date "2024-01-18":
>     * Gross profit: 3219.70 - 2202.08 - 1000 = 1017.62
>     * Profit in percentage: (1017.62) / (2202.08 + 1000) * 100 = 33.91%
> 
> * Cycle of date "2024-01-31":
>     * Gross profit: 3236.95 - 3219.70 - 500 = 17.25
>     * Profit in percentage: (17.25) / (3219.70 + 500) * 100 = 0.54%

-------------------------------------------


> **user**: Take this as base for the following questions: {"investments": [{"id": "INTER CONSERVADOR FIRF CP", "cycles": [{"value": "887.25", "date": "2023-12-17"}]}, {"id": "INTER CORPORATE FIRF CP", "cycles": [{"value": "336.89", "date": "2023-12-17"}]}, {"id": "BUTIA TOP CR\u00c9DITO PRIVADO FIC FIRF", "cycles": [{"value": "565.46", "date": "2023-12-17"}]}], "applications": [{"id": "INTER CONSERVADOR FIRF CP", "cycles": [{"value": "500", "date": "2024-01-31"}]}, {"id": "INTER CORPORATE FIRF CP", "cycles": []}, {"id": "BUTIA TOP CR\u00c9DITO PRIVADO FIC FIRF", "cycles": [{"value": "1000", "date": "2024-02-29"}, {"value": "1000", "date": "2024-03-12"}]}], "returns": [{"id": "INTER CONSERVADOR FIRF CP", "cycles": [{"value": "887.49", "date": "2023-12-20"}, {"value": "890.25", "date": "2023-12-29"}, {"value": "896.02", "date": "2024-01-18"}, {"value": "899.84", "date": "2024-01-31"}]}, {"id": "INTER CORPORATE FIRF CP", "cycles": [{"value": "337.19", "date": "2023-12-20"}, {"value": "338.08", "date": "2023-12-29"}, {"value": "340.08", "date": "2024-01-18"}, {"value": "341.53", "date": "2024-01-31"}]}, {"id": "BUTIA TOP CR\u00c9DITO PRIVADO FIC FIRF", "cycles": [{"value": "565.96", "date": "2023-12-20"}, {"value": "567.71", "date": "2023-12-29"}, {"value": "571.26", "date": "2024-01-18"}, {"value": "573.98", "date": "2024-01-31"}]}]}

-------------------------------------------


> **model**: **INTER CONSERVADOR FIRF CP**
> 
> * Cycle of date "2024-01-18":
>     * Gross profit: 896.02 - 890.25 = 5.77
>     * Profit in percentage: (5.77) / (890.25 + 500) * 100 = 0.65%
> 
> * Cycle of date "2024-01-31":
>     * Gross profit: 899.84 - 896.02 = 3.82
>     * Profit in percentage: (3.82) / (896.02 + 500) * 100 = 0.43%
> 
> **INTER CORPORATE FIRF CP**
> 
> * No applications were made to this fund, so there is no profit.
> 
> **BUTIA TOP CRÉDITO PRIVADO FIC FIRF**
> 
> * Cycle of date "2024-01-18":
>     * Gross profit: 571.26 - 567.71 - 1000 = -438.45
>     * Profit in percentage: (-438.45) / (567.71 + 1000) * 100 = -33.33%
> 
> * Cycle of date "2024-01-31":
>     * Gross profit: 573.98 - 571.26 - 1000 = -426.28
>     * Profit in percentage: (-426.28) / (571.26 + 1000) * 100 = -32.28%

-------------------------------------------


In [8]:
prompt = "Show me a summary of the data provided"
while prompt.lower() != "thanks":
  response = chat_bot.send_message(prompt)
  display(to_markdown(f'**VTR**: {response}'))
  prompt = input("Ask to VTR: ")

> **VTR**: **Summary of Investment Data**
> 
> | Fund | Cycle Date | Gross Profit | Profit in Percentage |
> |---|---|---|---|
> | INTER CONSERVADOR FIRF CP | 2024-01-18 | 5.77 | 0.65% |
> | INTER CONSERVADOR FIRF CP | 2024-01-31 | 3.82 | 0.43% |
> | INTER CORPORATE FIRF CP | N/A | N/A | N/A |
> | BUTIA TOP CRÉDITO PRIVADO FIC FIRF | 2024-01-18 | -438.45 | -33.33% |
> | BUTIA TOP CRÉDITO PRIVADO FIC FIRF | 2024-01-31 | -426.28 | -32.28% |
> 
> **Notes:**
> 
> * INTER CORPORATE FIRF CP has no applications, so there is no profit.
> * BUTIA TOP CRÉDITO PRIVADO FIC FIRF has negative profits in both cycles, indicating a loss.

ValueError: content must not be empty